#### **🧩 📘 Code Cell 1 — Configure Grok Client (with Headings + Comments)**

In [12]:
# ============================================================
# 📘 SECTION 1 — Import Required Libraries
# ------------------------------------------------------------
# Why?
#   - os: Interact with operating system (read env variables)
#   - dotenv: Load API keys from .env files securely
#   - OpenAI: Used because Groq follows the OpenAI-compatible API format
# ============================================================

import os
from dotenv import load_dotenv
from openai import OpenAI

# ============================================================
# 📘 SECTION 2 — Load Environment Variables (.env.dev)
# ------------------------------------------------------------
# Why?
#   - Keeps API keys OUT of source code
#   - Allows switching between environments:
#       dev / uat / prod
#   - Uses python-dotenv to read envs/.env.dev
# ============================================================

load_dotenv("../../../envs/.env.dev")

# ============================================================
# 📘 SECTION 3 — Read the Groq API Key from Environment
# ------------------------------------------------------------
# Why?
#   - GROQ_API_KEY is stored safely in .env.dev
#   - Using os.getenv ensures security + flexibility
#   - If the key is missing, we raise an error immediately
# ============================================================

groq_api_key = os.getenv("GROQ_API_KEY")

if not groq_api_key:
    raise RuntimeError(
        "❌ GROQ_API_KEY is missing. Please verify it exists inside envs/.env.dev"
    )

# ============================================================
# 📘 SECTION 4 — Create the Groq Client (OpenAI-compatible)
# ------------------------------------------------------------
# Why?
#   - Groq uses OpenAI-style API endpoints
#   - base_url MUST be set to https://api.groq.com/openai/v1
#   - After this, we can use:
#         client.chat.completions.create(...)
#   - This client object will be reused in all other notebook cells
# ============================================================

client = OpenAI(
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1"
)

print("✅ Groq client configured successfully and ready to use.")

✅ Groq client configured successfully and ready to use.


#### **First Grok Chat Request (Hello World LLM request)**

In [13]:
# ============================================================
# 📘 SECTION 5 — First Chat Request to Groq (Hello World)
# ------------------------------------------------------------
# Goal:
#   - Verify that the Groq client works end-to-end
#   - Send a simple question to the model
#   - Receive and print the assistant's reply
# ============================================================

# 1️⃣ Build the messages list (conversation context)
messages = [
    {
        "role": "system",
        "content": "You are a friendly Python tutor. Explain things in simple language."
    },
    {
        "role": "user",
        "content": "Hello Groq! This is my first request. Please introduce yourself in 2–3 lines."
    }
]

# 2️⃣ Send the chat completion request to Groq
#    Using a supported model: llama-3.1-8b-instant
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=messages
)

# 3️⃣ Extract the assistant's reply
#    IMPORTANT:
#    - response.choices[0].message is an object (ChatCompletionMessage)
#    - So we must use `.content` (attribute), NOT ["content"] (dict style)
assistant_reply = response.choices[0].message.content

# 4️⃣ Print reply
print("🤖 Groq says:\n")
print(assistant_reply)

🤖 Groq says:

I'm Groq, your friendly Python tutor. I'm here to help you learn and explore the amazing world of Python. I'll try to make coding fun and easy to understand.


#### **Inspect the Raw Response Object**

In [14]:
# ============================================================
# 📘 SECTION 6 — Inspecting the Raw Response Object
# ------------------------------------------------------------
# Why?
#   - To see the full structure returned by the LLM.
#   - This helps us understand:
#       * where the model's reply lives
#       * how choices[] is structured
#       * how we might access metadata later (tokens, model, etc.)
# ============================================================

# 🧩 1) Build a simple messages list for testing
#    Why?
#      - We send a short, clear question so the response object
#        is easy to read and understand.

messages = [
    {
        "role":"system",
        "content":"You are a helpful assistant who explains things simply:" 
    },
    {
        "role":"user",
        "content":"What is a response object in the context of LLM APIs? Explain briefly."
    }
]

# 🧩 2) Send a chat completion request to Groq
#    Why?
#      - Same pattern as before:
#          client.chat.completions.create(...)
#      - We use the same model as in Section 5.
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=messages
)

# 🧩 3) Print the entire response object
#    Why?
#      - For learning, we want to see everything Groq returns.
#      - In real applications, we wouldn't print this every time.

print("======= RAW RESPONSE OBJECT =======")
print(response)

======= RAW RESPONSE OBJECT =======
ChatCompletion(id='chatcmpl-b64537d8-5681-4560-a892-fdfea48fff9c', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="In the context of Large Language Model (LLM) APIs, a response object is a data structure that contains the output of the model's processing. It typically includes:\n\n1. The generated text or response to the input query or prompt\n2. Metadata such as timestamp, model version, and model configuration\n3. Optional fields such as error messages, warnings, or confidence scores\n\nThe response object varies in format depending on the specific LLM API being used, but it generally provides the output of the model in a structured and easily consumable way.", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1764707117, model='llama-3.1-8b-instant', object='chat.completion', service_tier='on_demand', system_fingerprint='fp_ff2b098aaf',

#### **Extract Key Fields from the Response Object**

In [16]:
# ============================================================
# 📘 SECTION 7 — Extracting Important Fields from Response
# ------------------------------------------------------------
# Goal:
#   - Understand how to read specific parts of the response:
#       * Model name
#       * Assistant role
#       * Assistant message (reply)
#       * Finish reason
#       * Token usage (if available)
#
# Note:
#   - This assumes 'response' already exists from SECTION 6.
#   - If not, re-run SECTION 6 before running this cell.
# ============================================================
# 🧩 1) Extract the model name
#    Why?
#      - Useful for logging, debugging, and knowing which LLM handled the request.

model_name = response.model

# 🧩 2) Extract the first choice (index 0)
#    Why?
#      - Most calls only care about the first suggested answer.

first_choice = response.choices[0]

# 🧩 3) Extract the assistant's message role and content
#    Why?
#      - role  → usually "assistant"
#      - content → actual reply text from the model

assistant_role = first_choice.message.role
assistant_content = first_choice.message.content

# 🧩 4) Extract the finish reason
#    Why?
#      - Tells us WHY the model stopped generating:
#          * "stop"        → completed naturally
#          * "length"      → hit max_tokens limit
#          * "content_filter" → blocked by safety filter (in some providers)

finish_reason = first_choice.finish_reason

# 🧩 5) Extract token usage (if available)
#    Why?
#      - Helps us understand cost and length of prompts/responses.
#      - Some providers may not always return usage; we handle that safely.

usage_info = getattr(response,"usage",None)

if usage_info:
    prompt_tokens = usage_info.prompt_tokens
    completion_tokens = usage_info.completion_tokens
    total_tokens = usage_info.total_tokens
else:
    prompt_tokens = completion_tokens = total_tokens = None

# 🧩 6) Print everything in a clean, readable way

print("===== 🔎 RESPONSE SUMMARY =====")
print(f"Model Used    :{model_name}")
print(f"Assistant role :{assistant_role}")
print(f"Finish reason  :{finish_reason}")
print()
print({"----- 🧠 Assistant Reply -----"})
print(assistant_content)
print()

if usage_info:
    print("----- 📊 Token Usage -----")
    print(f"Prompt tokens    :{prompt_tokens}")
    print(f"completion_tokens   :{completion_tokens}")
    print(f"Total Tokens   :{total_tokens}")
else:
    print("Token usage information not provided by this response.")

===== 🔎 RESPONSE SUMMARY =====
Model Used    :llama-3.1-8b-instant
Assistant role :assistant
Finish reason  :stop

{'----- 🧠 Assistant Reply -----'}
In the context of Large Language Model (LLM) APIs, a response object is a data structure that contains the output of the model's processing. It typically includes:

1. The generated text or response to the input query or prompt
2. Metadata such as timestamp, model version, and model configuration
3. Optional fields such as error messages, warnings, or confidence scores

The response object varies in format depending on the specific LLM API being used, but it generally provides the output of the model in a structured and easily consumable way.

----- 📊 Token Usage -----
Prompt tokens    :61
completion_tokens   :112
Total Tokens   :173
